# Introduction

**Définition d'une variable catégorielle** ([Source Wikipedia](https://fr.wikipedia.org/wiki/Variable_cat%C3%A9gorielle))  

En statistique, une variable qualitative, une variable catégorielle, ou bien un facteur est une variable qui prend pour valeur des modalités, des catégories ou bien des niveaux, par opposition aux variables quantitatives qui mesurent sur chaque individu une quantité. 

**Type de variable catégorielle**   

On distingue deux types de variable catégorielle :

* Variables ordinales : une variable catégorielle est dite ordinale lorsque ses modalités (catégories) peuvent être classées dans un ordre spécifique ou dans un ordre naturel quelconque. 
  * Exemple : On peut définir une variable ordinale « intensité » qui prend les valeurs suivantes : très faible, faible, fort, très fort
* Variables nominales : Contrairement aux variables ordinales, les catégories d'une variable nominale ne suivent pas un ordre naturel.
  * Exemple : la couleur des yeux

La gestion des variables catégorielles peut un rôle important dans les performances d'un modèle statistique. Elle dépend du nombre de catégories de la variable.

# A. Variables ordinales

Pour les variables ordinales, on peut les convertir en entier stocké dans une seule colonne en respectant l'ordre. On peut spécifier les entiers à utiliser pour remplacer les modalités. La méthode est efficace lorsque la dépendance être la variable et la cible est linéaire.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from category_encoders.ordinal import OrdinalEncoder

data = pd.DataFrame({'index': range(5), 
                     'note': ['tres agreable', 'agreable', 'neutre', 'desagreable', 'tres desagreable']})

mappings_note = {
    'tres desagreable': -2,
    'desagreable': -1,
    'neutre': 0,
    'agreable': 1,
    'tres agreable':2
}

model = OrdinalEncoder(mapping=[{'col': 'note', 'mapping': mappings_note}]).fit(data)
model.transform(data)

,index,note
0,0,2
1,1,1
2,2,0
3,3,-1
4,4,-2


In [2]:
from category_encoders.ordinal import OrdinalEncoder

data = pd.DataFrame({'index': range(5), 
                     'note': ['tres agreable', 'agreable', 'neutre', 'desagreable', 'tres desagreable']})


model = OrdinalEncoder(cols=['note']).fit(data)
model.transform(data)

,index,note
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5


# B. Variables nominales

### 1. Losque la variable nominale n'a pas beaucoup de modalités

**One-Hot Encoding**

Le One hot encoding est une représentation binaire d'une variable catégorielle. Chaque catégorie devient une colonne qui vaut 0 et 1.

Cette technique est plus adaptée aux modèles basés sur les arbres binaires. Cependant elle peut poser des problèmes de convergences pour les méthodes nécessitant des matrices inversibles.

In [3]:
from sklearn.preprocessing import OneHotEncoder
data = pd.DataFrame({'ID_PERS': range(14), 
                     'COULEURS_YEUX': ['bleus', 'rouges', 'marron',
                                       'noirs', 'gris', 'verts', 'bleus', 
                                       'noirs', 'rouges', 'noirs', 'gris', 
                                       'rouges', 'verts', 'verts']})

model =  OneHotEncoder().fit(data[['COULEURS_YEUX']])

columns = ['FEATURES_' + col.upper() for col in model.categories_[0]]
data_ =  pd.DataFrame(model.transform(data[['COULEURS_YEUX']]).toarray(), columns=columns)
data = data.join(data_, how='left')
data.head().transpose()

,0,1,2,3,4
ID_PERS,0,1,2,3,4
COULEURS_YEUX,bleus,rouges,marron,noirs,gris
FEATURES_BLEUS,1.0,0.0,0.0,0.0,0.0
FEATURES_GRIS,0.0,0.0,0.0,0.0,1.0
FEATURES_MARRON,0.0,0.0,1.0,0.0,0.0
FEATURES_NOIRS,0.0,0.0,0.0,1.0,0.0
FEATURES_ROUGES,0.0,1.0,0.0,0.0,0.0
FEATURES_VERTS,0.0,0.0,0.0,0.0,0.0


**Dummy coding**  

La librairie pandas propose une méthode pour gérer des variables catégorielles qui est peu différente de `sklearn.preprocessing.OneHotEncoder`:
* `pandas.get_dummies` autorise la suppression d'une modalité (colonne) dans la représentation binaire pour diminuer les effets de colinéarité. Cette méthode qui consiste à représenter la variable catégorielle avec `k-1` colonnes binaires est aussi appelé *Dummy coding*.
* `sklearn.preprocessing.OneHotEncoder` est un estimateur donc plus adapté aux pipelines

In [4]:
data = pd.DataFrame({'ID_PERS': range(14), 
                     'COULEURS_YEUX': ['bleus', 'rouges', 'marron',
                                         'noirs', 'gris', 'verts', 'bleus', 
                                         'noirs', 'rouges', 'noirs', 'gris', 
                                         'rouges', 'verts', 'verts']})

print(sorted(data['COULEURS_YEUX'].unique()))

data = pd.get_dummies(data, columns=['COULEURS_YEUX'], prefix='FEATURES', drop_first=False, dtype=np.float32)
data.columns = data.columns.str.upper()
data.head(5).transpose()

['bleus', 'gris', 'marron', 'noirs', 'rouges', 'verts']


,0,1,2,3,4
ID_PERS,0.0,1.0,2.0,3.0,4.0
FEATURES_BLEUS,1.0,0.0,0.0,0.0,0.0
FEATURES_GRIS,0.0,0.0,0.0,0.0,1.0
FEATURES_MARRON,0.0,0.0,1.0,0.0,0.0
FEATURES_NOIRS,0.0,0.0,0.0,1.0,0.0
FEATURES_ROUGES,0.0,1.0,0.0,0.0,0.0
FEATURES_VERTS,0.0,0.0,0.0,0.0,0.0


**Effect Coding**

Le dummy coding supprime une colonne de la représentation de binaire de la variable d'une variable catégorielle. Les individus qui possédent la modalité correspondant à la colnne binaire supprimée, auront des lignes remplies de zéros dans cette réprésentation. L'effect coding consiste à remplacer ces zéros par -1.

Lorsqu'on a plusieurs variables catégorielles, l'effect coding et le dummy coding peuvent donner des résultats similaires.
En revanche lorsqu'on a un nombre très faibles de variable catégorielle, utiliser l'effect coding est une bonne option surtout pour les méthodes comme la régression linéaire.

In [5]:
data = pd.DataFrame({'ID_PERS': range(14), 
                     'COULEURS_YEUX': ['bleus', 'rouges', 'marron',
                                         'noirs', 'gris', 'verts', 'bleus', 
                                         'noirs', 'rouges', 'noirs', 'gris', 
                                         'rouges', 'verts', 'verts']})

print(sorted(data['COULEURS_YEUX'].unique()))

data = pd.get_dummies(data, columns=['COULEURS_YEUX'], prefix='FEATURES', drop_first=True, dtype=np.float32)
data.columns = data.columns.str.upper()
data.head(5).transpose()
features = [col for col in data.columns if col not in ['ID_PERS']]
index = data[data[features].sum(axis=1) == 0].index
data.loc[index, features] = -1

data

['bleus', 'gris', 'marron', 'noirs', 'rouges', 'verts']


,ID_PERS,FEATURES_GRIS,FEATURES_MARRON,FEATURES_NOIRS,FEATURES_ROUGES,FEATURES_VERTS
0,0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,0.0,0.0,0.0,1.0,0.0
2,2,0.0,1.0,0.0,0.0,0.0
3,3,0.0,0.0,1.0,0.0,0.0
4,4,1.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,1.0
6,6,-1.0,-1.0,-1.0,-1.0,-1.0
7,7,0.0,0.0,1.0,0.0,0.0
8,8,0.0,0.0,0.0,1.0,0.0
9,9,0.0,0.0,1.0,0.0,0.0


### 2. Losque la variable nominale a beaucoup de modalités

**Target Encoding**

Le Target Encoding consiste à remplacer chaque modalité par la moyenne de la variable cible pour les individus possédant cette modalité. L'avantage de cette méthode est qu'elle n'augmente pas le nombre de variable. Cependant cette technique a des limites car elle peut :   
* Conduire à un sur-apprentissage
* Entraîner une éventuelle perte d'information car des catégories différentes peuvent remplacées par la même valeur

In [6]:
from category_encoders import TargetEncoder
import pandas as pd
from sklearn.datasets import load_boston
from IPython.display import display

pd.options.display.float_format = '{:,.2f}'.format
bunch = load_boston()
y = bunch.target

data = pd.DataFrame(bunch.data, columns=bunch.feature_names)
data['RAD'] = data['RAD'].astype(int).astype('category')
data['CHAS'] = data['CHAS'].astype(int).astype('category')
display(data.head())

model = TargetEncoder(cols=['CHAS', 'RAD']).fit(data, y)
data = model.transform(data)
display(data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,0,0.54,6.58,65.20,4.09,1,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,0,0.47,6.42,78.90,4.97,2,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,0,0.47,7.18,61.10,4.97,2,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,0,0.46,7.00,45.80,6.06,3,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,0,0.46,7.15,54.20,6.06,3,222.00,18.70,396.90,5.33


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,22.09,0.54,6.58,65.20,4.09,24.36,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,22.09,0.47,6.42,78.90,4.97,26.83,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,22.09,0.47,7.18,61.10,4.97,26.83,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,22.09,0.46,7.00,45.80,6.06,27.93,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,22.09,0.46,7.15,54.20,6.06,27.93,222.00,18.70,396.90,5.33


**Count Encoding**

Cette technique remplace chaque modalité par sa fréquence. Comme le Target encoding, elle peut aussi entrainer une perte d'information.

In [7]:
from category_encoders import CountEncoder
import pandas as pd
from sklearn.datasets import load_boston
from IPython.display import display

bunch = load_boston()
y = bunch.target
data = pd.DataFrame(bunch.data, columns=bunch.feature_names)
data['RAD'] = data['RAD'].astype(int).astype('category')
data['CHAS'] = data['CHAS'].astype(int).astype('category')
display(data.head())

model = CountEncoder(cols=['CHAS', 'RAD'], normalize=False).fit(data)
data = model.transform(data)
display(data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,0,0.54,6.58,65.20,4.09,1,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,0,0.47,6.42,78.90,4.97,2,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,0,0.47,7.18,61.10,4.97,2,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,0,0.46,7.00,45.80,6.06,3,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,0,0.46,7.15,54.20,6.06,3,222.00,18.70,396.90,5.33


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,471,0.54,6.58,65.20,4.09,20,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,471,0.47,6.42,78.90,4.97,24,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,471,0.47,7.18,61.10,4.97,24,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,471,0.46,7.00,45.80,6.06,38,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,471,0.46,7.15,54.20,6.06,38,222.00,18.70,396.90,5.33


**Feature Hashing**

Chaque catégorie est transformée en une chaîne de caractères de longueur fixe. Cette méthode permet de reduire le nombre de colonnes de la représentation binaire d'une variable catégorielle en ajustant le hash de sorte que plusieurs catégories seront encodées de la même manière.

In [8]:
from category_encoders.hashing import HashingEncoder
import pandas as pd
from sklearn.datasets import load_boston
from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format

bunch = load_boston()
y = bunch.target
data = pd.DataFrame(bunch.data, columns=bunch.feature_names)
data['RAD'] = data['RAD'].astype(int).astype('category')
data['CHAS'] = data['CHAS'].astype(int).astype('category')
display(data.head())

model = HashingEncoder(cols=['CHAS', 'RAD'], n_components=4).fit(data, y)
data = model.transform(data)
display(data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,0,0.54,6.58,65.20,4.09,1,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,0,0.47,6.42,78.90,4.97,2,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,0,0.47,7.18,61.10,4.97,2,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,0,0.46,7.00,45.80,6.06,3,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,0,0.46,7.15,54.20,6.06,3,222.00,18.70,396.90,5.33


,col_0,col_1,col_2,col_3,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT
0,0,0,1,1,0.01,18.00,2.31,0.54,6.58,65.20,4.09,296.00,15.30,396.90,4.98
1,1,0,1,0,0.03,0.00,7.07,0.47,6.42,78.90,4.97,242.00,17.80,396.90,9.14
2,1,0,1,0,0.03,0.00,7.07,0.47,7.18,61.10,4.97,242.00,17.80,392.83,4.03
3,0,0,1,1,0.03,0.00,2.18,0.46,7.00,45.80,6.06,222.00,18.70,394.63,2.94
4,0,0,1,1,0.07,0.00,2.18,0.46,7.15,54.20,6.06,222.00,18.70,396.90,5.33


**Entity Embedding**

Cette méthode basée sur les réseaux de neurones s'inspire de modèles comme `word2vec`. Elle propose une représentation vectorielle des modalités d'une variable catégorielle. Cette méthode permet réduire la dimension mais ajoute des difficultés supplémentaires sur l'interprétabilité du modèle.

L'exemple ci-dessous tente de modéliser cette technique. Chaque modalité sera représentée par un scalaire (vecteur de taille 1).

In [9]:
import category_encoders.utils as util
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


class EmbeddingEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, column=None, metrics=['mse']):
        self.column = column
        self._model = None
        self._is_model_fitted = False
        self._labelEncoder = None
        self.metrics = metrics

    def fit(self, X, y):
        X_copy = X.copy(deep=True)
        X_copy = util.convert_input(X_copy)

        self._labelEncoder = LabelEncoder().fit(X_copy[self.column])
        X_copy[self.column] = self._labelEncoder.transform(X_copy[self.column])
        nb_class = len(self._labelEncoder.classes_)

        self._model = Sequential(
            [
                Embedding(input_dim=nb_class, 
                          output_dim= 1 + int(nb_class/3), 
                          input_length=1, 
                          name="embedding"),
                Flatten(),
                Dense(7, activation="relu"),
                Dense(4, activation="relu"),
                Dense(1)])
        
        self._model.compile(loss="mse", optimizer="adam", metrics=self.metrics)
        self._model.fit(x=X_copy[self.column],
                        y=y,
                        epochs=50,
                        batch_size=5,
                        verbose=0)

        self._is_model_fitted = True
        return self
  
    def transform(self, X):
        if not self._is_model_fitted:
            raise ValueError(
                "Entrainer l'encodeur avant de l'utiliser"
            )

        if not self.column:
            return X
        
        X_transformed = X.copy(deep=True)
        X_transformed = util.convert_input(X_transformed)
        X_transformed[self.column] = self._labelEncoder.transform(X_transformed[self.column])     
        X_transformed[self.column] = self._model.predict(X_transformed[self.column])

        return X_transformed

In [10]:
import pandas as pd
from sklearn.datasets import load_boston
from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format

bunch = load_boston()
y = bunch.target
data = pd.DataFrame(bunch.data, columns=bunch.feature_names)
display(data.head())

model = EmbeddingEncoder(column='RAD').fit(data, y)
data = model.transform(data)
display(data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,0.00,0.54,6.58,65.20,4.09,1.00,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,0.00,0.47,6.42,78.90,4.97,2.00,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,0.00,0.47,7.18,61.10,4.97,2.00,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,0.00,0.46,7.00,45.80,6.06,3.00,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,0.00,0.46,7.15,54.20,6.06,3.00,222.00,18.70,396.90,5.33


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01,18.00,2.31,0.00,0.54,6.58,65.20,4.09,24.36,296.00,15.30,396.90,4.98
1,0.03,0.00,7.07,0.00,0.47,6.42,78.90,4.97,26.92,242.00,17.80,396.90,9.14
2,0.03,0.00,7.07,0.00,0.47,7.18,61.10,4.97,26.92,242.00,17.80,392.83,4.03
3,0.03,0.00,2.18,0.00,0.46,7.00,45.80,6.06,28.01,222.00,18.70,394.63,2.94
4,0.07,0.00,2.18,0.00,0.46,7.15,54.20,6.06,28.01,222.00,18.70,396.90,5.33


Sources :   
[Category Encoders](http://contrib.scikit-learn.org/category_encoders/index.html)       
[Deep embedding’s for categorical variables (Cat2Vec)](https://towardsdatascience.com/deep-embeddings-for-categorical-variables-cat2vec-b05c8ab63ac0)    
[FAQ: WHAT IS EFFECT CODING?](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faqwhat-is-effect-coding/)